In [1]:
import pandas as pd
import numpy as np
import glob
import os
import datetime
import re
from datetime import datetime

# %cd /Volumes/SEB_USB/Season_10/

In [2]:
def get_date_from_string(text):
    match = re.search(r'\d{4}-\d{2}-\d{2}', text)
    date = datetime.strptime(match.group(), '%Y-%m-%d').date()
    return date


# --------------------------------------------------
def get_ps2_dataframe(csv_path):
    df_list = []

    for csv in glob.glob(csv_path):
        date_df = pd.read_csv(csv)
        date = get_date_from_string(os.path.basename(csv))
        date_df['date'] = pd.to_datetime(date)
        df_list.append(date_df)  

    ps2_df = pd.concat(df_list).drop('Unnamed: 0', axis=1)

    return ps2_df


# --------------------------------------------------
def clean_plot_column_rgb_flir(rgb_flir_merged):
    
    rgb_flir_merged['plot'] = rgb_flir_merged['plot'].str.split('_', expand=True)[6].astype(str).str.zfill(2) + rgb_flir_merged['plot'].str.split('_', expand=True)[8].astype(str).str.zfill(2)

    return rgb_flir_merged


# --------------------------------------------------
def clean_plot_column_ps2(ps2_df):

    ps2_df['plot'] = ps2_df['Plot'].str.split(' ', expand=True)[6].astype(str).str.zfill(2) + ps2_df['Plot'].str.split(' ', expand=True)[8].astype(str).str.zfill(2)
    ps2_df = ps2_df.drop('Plot', axis=1)

    return ps2_df


# --------------------------------------------------
def get_3d_dataframe(csv_path):

    df_list = []

    for csv in glob.glob(csv_path):
        date_df = pd.read_csv(csv)
        date = get_date_from_string(os.path.basename(csv))
        date_df['date'] = pd.to_datetime(date)
        df_list.append(date_df) 

    three_df = pd.concat(df_list).drop('Unnamed: 0', axis=1)

    return three_df

# RGB & FLIR

### Open data

In [3]:
thermal = pd.read_csv('Haiquan/s10_flir.csv').drop('treatment', axis=1)
rgb = pd.read_csv('Haiquan/stereoTop_full_season_clustering.csv')

### Convert date string to datetime

In [4]:
thermal['date'] = pd.to_datetime(thermal['date'])
rgb['date'] = pd.to_datetime(rgb['date'])

### Set shared index

In [5]:
rgb = rgb.set_index(['date', 'plant_name'])
thermal = thermal.set_index(['date', 'plant_name'])

### Join RGB and FLIR

In [6]:
rgb_flir_merged = rgb.join(thermal, how='outer').reset_index()

In [7]:
rgb_flir_merged = clean_plot_column_rgb_flir(rgb_flir_merged)

# RGB|FLIR & PSII

### Open PSII data

In [8]:
ps2_df = get_ps2_dataframe('./Haiquan/PS2/*.csv')
ps2_df = clean_plot_column_ps2(ps2_df)

### Prepare RGB|FLIR & PSII for merging

In [9]:
ps2_df = ps2_df.set_index(['date', 'plot'])
rgb_flir_merged = rgb_flir_merged.set_index(['date', 'plot'])

### Merge RGB|FLIR and PSII

In [10]:
merged = rgb_flir_merged.join(ps2_df, how='outer').reset_index()

### Save merged file

In [11]:
merged.to_csv('season10_lettuce_gantry_data.csv')

# RGB|FLIR|PSII & 3D

### Open 3D data

In [12]:
three_df = get_3d_dataframe('tda_volumes_full_resolution/*.csv')

### Merge RGB|FLIR|PSII and 3D

In [13]:
final_merged = merged.set_index(['date', 'plant_name']).join(three_df.set_index(['date', 'plant_name']), how='outer').reset_index()

### Save final merged file

In [14]:
final_merged.to_csv('season10_lettuce_gantry_data_full.csv')

In [19]:
final_merged[final_merged['plot'].isna()==True][final_merged['bounding_area_m2'].isna()==True]

<ipython-input-19-1b72ff6597d1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  final_merged[final_merged['plot'].isna()==True][final_merged['bounding_area_m2'].isna()==True]


,date,plant_name,plot,treatment,genotype,bounding_area_m2,double_lettuce,roi_temp,quartile_1,mean,...,amplitude_betti_2,amplitude_silhouette_0,amplitude_silhouette_1,amplitude_silhouette_2,amplitude_heat_0,amplitude_heat_1,amplitude_heat_2,amplitude_persistence_image_0,amplitude_persistence_image_1,amplitude_persistence_image_2
354971,2020-01-21,PI171674_101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.557676,0.004507,0.000972,0.000336,150.604815,42.204181,1.667405,647.647403,322.650328,20.985729
354972,2020-01-21,PI171674_11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.768995,0.004262,0.000996,0.000170,161.507735,39.729666,1.090882,695.873834,317.546014,22.628201
354973,2020-01-21,PI171674_110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.358889,0.004288,0.000855,0.000093,191.498816,50.259362,0.607415,726.347458,330.222604,7.671277
354974,2020-01-21,PI171674_118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.707392,0.004466,0.001082,0.000235,227.028594,44.896337,2.510646,944.547410,469.901286,32.485606
354975,2020-01-21,PI171674_119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.260509,0.004233,0.000986,0.000164,173.822029,51.095550,0.285258,693.621670,330.908111,6.048019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766913,2020-03-02,Xanadu_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.121534,0.004256,0.001076,0.000091,241.261899,74.729223,1.975541,1026.379123,563.860036,31.403703
766920,2020-03-02,Xanadu_70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.445943,0.004133,0.001259,0.000215,196.653605,29.407944,0.489602,668.258105,253.386819,5.984819
766924,2020-03-02,Xanadu_81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.350555,0.004200,0.001438,0.000100,169.963132,31.831082,0.699096,635.486855,292.805433,11.233126
766927,2020-03-02,Xanadu_85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.943337,0.004191,0.000995,0.000177,209.727873,66.337438,1.562622,875.947046,523.416652,25.938094
